In [16]:
%load_ext autoreload
from coffea import util, processor
from msdprocessor5 import msdProcessor

from coffea.nanoevents import NanoEventsFactory, BaseSchema, PFNanoAODSchema
import json
import distributed
import dask
import awkward as ak
import hist
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.animation import FuncAnimation, PillowWriter
import matplotlib.colors as mcolors
from mpl_toolkits.mplot3d import Axes3D
from hist import Hist
import dask_awkward
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
#Trying to loop through as many files as possible and stack histograms
directory_path = "/uscms/home/jennetd/nobackup/hbb-prod-modes/run3-triggers/data-mc/"

singlefile = ["/uscms/home/jennetd/nobackup/hbb-prod-modes/run3-triggers/data-mc/QCD_Pt470to600.root"]

#Create an array with all the files in /data-mc
fileset = []
for filename in os.listdir(directory_path):
    if filename.endswith(".root"):
        # Add the full file path to the fileset list
        fileset.append(os.path.join(directory_path, filename))
print(fileset)

['/uscms/home/jennetd/nobackup/hbb-prod-modes/run3-triggers/data-mc/ggF.root', '/uscms/home/jennetd/nobackup/hbb-prod-modes/run3-triggers/data-mc/VBF.root', '/uscms/home/jennetd/nobackup/hbb-prod-modes/run3-triggers/data-mc/WminusH.root', '/uscms/home/jennetd/nobackup/hbb-prod-modes/run3-triggers/data-mc/ZH.root', '/uscms/home/jennetd/nobackup/hbb-prod-modes/run3-triggers/data-mc/QCD_Pt470to600.root']


In [18]:
# Start summing events
file = fileset[0]
events = NanoEventsFactory.from_root({file: "/Events"},
    schemaclass=PFNanoAODSchema
    ).events()

In [19]:
#Add up all the events
# Write a processor with arguments for zcut and beta

result = msdProcessor().process(events, beta = 0.3, z_cut = 0.4, n = 1)

#beta = max beta to loop through
#z_cut = max z_cut to loop through
#n = how many histograms to draw - n=2 will divide beta and z_cut args into 2, n=3 into 3 and make three histograms... etc
#When n = 1 only one beta and z_cut are processed
print(result)

[{'sumw': dask.awkward<sum, type=Scalar, dtype=float32>, 'b00': Hist(Regular(40, 0, 400, name='msoftdrop', label='Jet $m_\\mathrm{softdrop}$ [GeV]'), storage=Weight()) # Sum: WeightedSum(value=0, variance=0) (has staged fills)}]


In [20]:
#Compute the matrix of results from the matrix of events

compute = dask.compute(result)

# save the output file
outfile = "histogram.coffea"
#util.save(compute, outfile)
print("saved " + outfile)

#print full compute

print(compute)

saved histogram.coffea
([{'sumw': 295.33975, 'b00': Hist(Regular(40, 0, 400, name='msoftdrop', label='Jet $m_\\mathrm{softdrop}$ [GeV]'), storage=Weight()) # Sum: WeightedSum(value=295.339, variance=139.338)}],)


In [21]:

# Define the number of betas to iterate over
n_betas = 1  # Adjust this to however many histograms you want
n_zcuts = n_betas

# Print something from compute for sanity check
print(compute[0][0])

# Convert the dict structure of full_compute into an array (compute_matrix)
compute_matrix = [[None for _ in range(n_zcuts)] for _ in range(n_betas)]
for beta in range(n_betas):
    for z_cut in range(n_zcuts):
        compute_matrix[beta][z_cut] = compute[0][0][f"b{beta}{z_cut}"]

# Define fixed axis limits
x_min, x_max = 0, 400  
y_min, y_max = 0, 300 

# Plotting the histograms
# We want exactly one figure per z-cut, overlaying all betas on the same figure
for z_cut in range(n_zcuts):
    fig, ax = plt.subplots()

    # For a given z_cut, loop over betas to overlay on the same axes
    for beta in range(n_betas):
        compute_matrix[beta][z_cut].plot1d(ax=ax, label=f"beta = {beta}")

    # Fix axis limits for consistency
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)

    # Set a title
    ax.set_title(f"z_cut = {z_cut} beta = {beta}")

    # Add legend and save the figure
    ax.legend()
    filename = f"plot_zcut{z_cut}.png"
    plt.savefig(filename, dpi=300)
    plt.close()


{'sumw': 295.33975, 'b00': Hist(Regular(40, 0, 400, name='msoftdrop', label='Jet $m_\\mathrm{softdrop}$ [GeV]'), storage=Weight()) # Sum: WeightedSum(value=295.339, variance=139.338)}


In [22]:
num_bins = h.axes[0].size
msoftdrop_bins = np.linspace(0, 400, num_bins + 1)  # Adjust 400 and number of bins as per your range
msoftdrop_axis = hist.axis.Regular(num_bins, msoftdrop_bins[0], msoftdrop_bins[-1], name="msoftdrop", label="Softdrop Mass")

# Define a categorical axis for beta values
beta_axis = hist.axis.IntCategory([0, 1, 2, 3, 4], name="beta", label="Beta")

# Create an empty 3D histogram with msoftdrop and beta axes
hist_3d = hist.Hist(msoftdrop_axis, beta_axis)

# Extract frequencies for each `beta` value and fill them into `hist_3d`
msoftdrop_vals = np.repeat(hist_3d.axes[0].centers, 5)  # repeat for each beta layer
beta_vals = np.tile(np.array([0, 1, 2, 3, 4]), num_bins)  # repeat beta values across msoftdrop bins

frequencies = np.concatenate([h.view(), i.view(), j.view(), k.view(), l.view()])

# Use the `fill` method to populate `hist_3d`
hist_3d.fill(msoftdrop=msoftdrop_vals, beta=beta_vals, weight=frequencies)

# Prepare data for 3D plotting
msoftdrop_vals, beta_vals = np.meshgrid(
    hist_3d.axes[0].centers,  # Centers of msoftdrop bins
    hist_3d.axes[1].centers   # Centers of beta bins
)
frequencies = hist_3d.view().T  # Transpose to match meshgrid shape

# Plotting in 3D
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Create a surface plot
surf = ax.plot_surface(
    msoftdrop_vals, beta_vals, frequencies,
    cmap="viridis", edgecolor='none'
)
ax.set_xlabel("Softdrop Mass (msoftdrop)")
ax.set_ylabel("Beta")
ax.set_zlabel("Frequency")
ax.set_title("3D Histogram of msoftdrop vs Beta")

# Add a color bar to show frequency values
fig.colorbar(surf, ax=ax, label="Frequency")
plt.show()

NameError: name 'h' is not defined

In [ ]:

#Plot the hist of an individual file
#Single Jet

h = compute[0][0]['b00']

fig, ax = plt.subplots()
h.plot1d(ax=ax,)  # Colors and Normalized Log Plot
plt.show()


In [ ]:
# Another individual jet
h = compute[2]['ExampleHistogram']

fig, ax = plt.subplots()
h.plot1d(ax=ax)  # For 2D histograms like pt vs eta
plt.show()


In [ ]:
# More Individual Jets
h = compute[1]['ExampleHistogram'][{'eta':sum}]

fig, ax = plt.subplots()
h.plot2d(ax=ax)  # For 2D histograms like pt vs eta
plt.show()


In [ ]:
#Make the first entry in the summation of the same type as the rest of the matrix, that is, 'ExampleHistogram' Type
summation = compute[1]['ExampleHistogram']

#Sum all the histograms of this same type to the first histogram
for index in compute[1:]:
    summation +=index['ExampleHistogram']

#Plot the summed hist 

fig, ax = plt.subplots()
summation.plot1d(ax=ax)
plt.show()


In [ ]:
#Full file Plot
summation[{'eta':sum}].plot1d()

# This sums over eta and draws pT

In [ ]:
i = compute[1]['ExampleHistogram']
i

In [ ]:
j = compute[2]['ExampleHistogram']
j

In [ ]:
k = compute[3]['ExampleHistogram']
k

In [ ]:
l = compute[4]['ExampleHistogram'][{'eta':sum,'pt':sum}]
l

In [ ]:
events.fields

In [ ]:
events['q1pt'].plot1d(overlay='wc')
plt.yscale('log')
#plt.ylim(1e-4, 1000)

plt.legend(ncol=3,bbox_to_anchor=(1.0,1.1));

In [ ]:
output['q2pt'].plot1d(overlay='wc')
plt.yscale('log')
#plt.ylim(1e-4, 1000)

plt.legend(ncol=3,bbox_to_anchor=(1.0,1.1));

In [ ]:
output['hpt'].plot1d(overlay='wc')
plt.yscale('log')
#plt.ylim(1e-4, 1000)

plt.legend(ncol=3,bbox_to_anchor=(1.0,1.1));

In [ ]:
output['detaqq'].plot1d(overlay='wc')
plt.yscale('log')
#plt.ylim(1e-4, 1000)

plt.legend(ncol=3,bbox_to_anchor=(1.0,1.1));

In [ ]:
output['dphiqq'].plot1d(overlay='wc')
plt.yscale('log')
#plt.ylim(1e-4, 1000)

plt.legend(ncol=3,bbox_to_anchor=(1.0,1.1));

In [ ]:
output['mqq'].plot1d(overlay='wc')
plt.yscale('log')
#plt.ylim(1e-4, 1000)

plt.legend(ncol=3,bbox_to_anchor=(1.0,1.1));